In [ ]:
import json
import pandas as pd
#!pip install openai -U
from openai import OpenAI
from enum import Enum
from typing import Optional, List
from pydantic import BaseModel
from pydantic import Field

In [ ]:
with open('../key/key.json') as f:
    k = json.load(f)['key']

In [ ]:
# create one time list of 10 human emotions
client = OpenAI(api_key=k)
class Chr_answer(BaseModel):
    Characteristics: List[str] = Field(None, description="Emotions as a list of strings")

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "List 30 human emotions that describe diffnon redundant human feelings."}
    ],
    response_format=Chr_answer,
)

#you can also procees from parsed eg json.loads(completion_song.choices[0].message.parsed.json())
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
emotions = out
print(emotions)

In [ ]:
# class Char(BaseModel): 
#     is_char: bool
#     characteristic: Optional[Characteristic] ?
#fiels is only defined for the first custom class?

#List 10 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."
client = OpenAI(api_key=k)

# class Characteristic(Enum):
#     Happiness = 'Happiness'
#     Sadness = 'Sadness'
#     Anger = 'Anger'
#     Fear = 'Fear'
#     Surprise = 'Surprise'
#     Disgust = 'Disgust'
#     Trust = 'Trust'
#     Anticipation ='Anticipation'
#     Joy ='Joy'
#     Love = 'Love'

# class BrandAttributes(BaseModel):
#     brand_name: str = Field(None, description=" name as str")
#     characteristics: List[Characteristic] = Field(None, description=" list of characteristics as str")

Characteristic = Enum('Characteristic', dict([(i, i) for i in emotions])) #MyEnumType = Enum('MyEnumType', myEnumStrings)

class BrandAttributes(BaseModel):
    brand_name: str = Field(None, description=" name as str")
    characteristics: List[Characteristic] = Field(None, description=" list of characteristics as str")

class BrandChars(BaseModel):
    a: List[BrandAttributes] = Field(None, description="List of BrandAttributes")
    
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "List 30 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."}
    ],
    response_format=BrandChars,
)

out = json.loads(completion.choices[0].message.content)
brand_emotions = out
out = list(out.values())[0]
df_brand = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','characteristic'])
df_brand

In [ ]:
#Take the example song, and list which one of the emotions in the given list are most closely associated with the song.")

client = OpenAI(api_key=k)

Characteristic = Enum('Characteristic', dict([(i, i) for i in emotions])) 

class SongAttributes(BaseModel):
    song_name: str
    song_lyrics_summary :str = Field(None, description="Summary as str; Don't state the exact lyrics.")
    characteristics: List[Characteristic]

class SongsChars(BaseModel):
    a: List[SongAttributes] = Field(None, description="List of song attributes.")

prompt = """
As prompt, take the song 'Black Balloon', list summary of its lyrics, and a list of the emotions from the given list that are most closely associated with the song.
"""

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    response_format=SongsChars,
)
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
df_song = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','lyrics_summary', 'characteristic'])
df_song

In [ ]:
#MultilabelBinarize brands and song
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=emotions)

df_brand2 = pd.DataFrame(mlb.fit_transform(df_brand['characteristic']), columns = emotions)
df_brand_mb = pd.merge(df_brand2, df_brand[['Name']], how ='left', left_index=True, right_index=True)
df_brand_mb.head()


In [ ]:
mlb = MultiLabelBinarizer(classes=emotions)

df_song2 = pd.DataFrame(mlb.fit_transform(df_song['characteristic']), columns = emotions)
df_song_mb = pd.merge(df_song2, df_song[['Name']], how ='left', left_index=True, right_index=True)
df_song_mb.head()

#song_e = list(song_emotions[0].values())[2]

#df_song = pd.DataFrame(mlb.fit_transform([song_e]), columns = emotions, index=[list(song_emotions[0].keys())[0]])
#df_song

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split 
  
# Create feature and target arrays 
X = df_brand_mb.drop(['Name'], axis =1)
y = df_brand_mb['Name'] 
  
# Split into training and test set 
X_train, X_test, y_train, y_test = train_test_split( 
             X, y, test_size = 0.2, random_state=42) 
  
knn = KNeighborsClassifier(n_neighbors=3) 
  
knn.fit(X_train, y_train) 
  
# Predict on dataset which model has not seen before 
print(knn.predict(X_test.values))

In [ ]:
X_test

In [ ]:
#take a look
df_brand_mb[df_brand_mb['Name'].isin(list(knn.predict(X_test.values)))]